1)2)3) - The below code does policy iteration and value iteration. The values for states after value iteration is same for value iteration (about 0.99) and policy iteration (about 0.98)

In [4]:
#Reference -- Introduction to AI by Russel exercise (AIMA)

import random
import operator
import numpy as np
orientations = [(1,0), (0, 1), (-1, 0), (0, -1)]#The different orientations are right, up,left, down
class MDP:

    def __init__(self, init, actlist, terminals, transitions={}, states=None, gamma=.9):
        if states:
            self.states = states
        else:
            self.states = set()
        self.init = init
        self.actlist = actlist
        self.terminals = terminals
        self.transitions = transitions
        self.gamma = gamma
        self.reward = {}

    def R(self, state):
        return self.reward[state]
    def T(self, state, action):
        if(self.transitions == {}):
            raise ValueError("Transition model is missing")
        else:
            return self.transitions[state][action]
    def actions(self, state):
        if state in self.terminals:
            return [None]
        else:
            return self.actlist

class GridMDP(MDP):

    def __init__(self, grid, terminals, init=(0, 0), gamma=.9):
        grid.reverse()  # because we want row 0 on bottom, not on top
        MDP.__init__(self, init, actlist=orientations,
                     terminals=terminals, gamma=gamma)
        self.grid = grid
        self.rows = len(grid)
        self.cols = len(grid[0])
        for x in range(self.cols):
            for y in range(self.rows):
                self.reward[x, y] = grid[y][x]
                if grid[y][x] is not None:
                    self.states.add((x, y))

    def T(self, state, action):
        if action is None:
            return [(0.0, state)]
        else:
            return [(0.8, self.go(state, action)),
                    (0.1, self.go(state,orientations[orientations.index(action)-1])),
                    (0.1, self.go(state, orientations[(orientations.index(action)+1) % len(orientations)]))]
    def go(self, state, direction):
        state1 = tuple(operator.add(state,direction))
        return state1 if state1 in self.states else state

    def to_grid(self, mapping):
        return list(reversed([[mapping.get((x, y), None)
                               for x in range(self.cols)]
                              for y in range(self.rows)]))

mdp = GridMDP([[0, 0, 0, +10],
                     [0, 0,  0, -10],
                     [0, None, 0, 0]],
                    terminals=[(0, 3), (1, 3)])
def argmin(seq, fn):
    best = seq[0]; best_score = fn(best)
    for x in seq:
        x_score = fn(x)
        if x_score < best_score:
            best, best_score = x, x_score
    return best
def argmax(seq, fn):
    return argmin(seq, lambda x: -fn(x))
def value_iteration(mdp, epsilon=0.001):

    U1 = {s: 0 for s in mdp.states}
    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    q = np.zeros((12,4))
    while True:
        U = U1.copy()
        delta = 0
#         qsa=
        for s in mdp.states:
#             qsa.append([sum([p * U[s1] for (p, s1) in T(s, a)])
#                                         for a in mdp.actions(s)])
#             new_q = qsa + alpha * (rsa + gamma * max(q[next_state, :]) - qsa)
            # Modified code to get q values for every state and action
            for a in mdp.actions(s):
                qsa = q[s,a]
                new_q = qsa+0.1*(R(s) + gamma * max([sum([p * U[s1] for (p, s1) in T(s, a)])
                                        for a in mdp.actions(s)])-qsa)
#             print(new_q)
                q[s, a] = new_q
            U1[s] = R(s) + gamma * max([sum([p * U[s1] for (p, s1) in T(s, a)])
                                        for a in mdp.actions(s)])
#             print(U1[s])
            delta = max(delta, abs(U1[s] - U[s]))
#         qsa=np.array(q)
        
        
        if delta < epsilon * (1 - gamma) / gamma:
            
                return U,q

def expected_utility(a, s, U, mdp):
    return sum([p * U[s1] for (p, s1) in mdp.T(s, a)])

def policy_iteration(mdp):
    U = {s: 0 for s in mdp.states}
    pi = {s: random.choice(mdp.actions(s)) for s in mdp.states}
    while True:
        U = policy_evaluation(pi, U, mdp)
        print("The values at every policy update",sum(U.values()))
        unchanged = True
        for s in mdp.states:
            a = max(mdp.actions(s), key=lambda a: expected_utility(a, s, U, mdp))
            if a != pi[s]:
                pi[s] = a
                unchanged = False
        if unchanged:
            return pi,U
def policy_evaluation(pi, U, mdp, k=20):
    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    for i in range(k):
        for s in mdp.states:
            U[s] = R(s) + gamma * sum([p * U[s1] for (p, s1) in T(s, pi[s])])
    return U


U,qsa= value_iteration(mdp)
P,Up =policy_iteration(mdp)
print("1. Values of states at convergence for value iteration",U)
print("2.qsa pair for every state", qsa)
print("3.The optimal values using policy iteration",Up)

('The values at every policy update', 0.0)
('The values at every policy update', 0.0)
('1. Values of states at convergence for value iteration', {(0, 1): 0.0, (1, 2): 0.0, (3, 2): 99.99897095698557, (0, 0): 0.0, (3, 0): 0.0, (3, 1): -99.99897095698557, (2, 1): 0.0, (1, 1): 0.0, (2, 0): 0.0, (2, 2): 0.0, (0, 2): 0.0})
('2.qsa pair for every state', array([[  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [-18.99982403,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ],


4) The below code is when discount factor is 0.5 . As it can be seen the values for the state is reduced. This could be because a lesser discount factor indicates that the future rewards must be given less importance.


In [4]:
import random
import operator
import numpy as np
orientations = [(1,0), (0, 1), (-1, 0), (0, -1)]#The different orientations are right, up,left, down
class MDP:

    def __init__(self, init, actlist, terminals, transitions={}, states=None, gamma=.5):
        if states:
            self.states = states
        else:
            self.states = set()
        self.init = init
        self.actlist = actlist
        self.terminals = terminals
        self.transitions = transitions
        self.gamma = gamma
        self.reward = {}

    def R(self, state):
        return self.reward[state]
    def T(self, state, action):
        if(self.transitions == {}):
            raise ValueError("Transition model is missing")
        else:
            return self.transitions[state][action]
    def actions(self, state):
        if state in self.terminals:
            return [None]
        else:
            return self.actlist

class GridMDP(MDP):

    def __init__(self, grid, terminals, init=(0, 0), gamma=.5):
        grid.reverse()  # because we want row 0 on bottom, not on top
        MDP.__init__(self, init, actlist=orientations,
                     terminals=terminals, gamma=gamma)
        self.grid = grid
        self.rows = len(grid)
        self.cols = len(grid[0])
        for x in range(self.cols):
            for y in range(self.rows):
                self.reward[x, y] = grid[y][x]
                if grid[y][x] is not None:
                    self.states.add((x, y))

    def T(self, state, action):
        if action is None:
            return [(0.0, state)]
        else:
            return [(1, self.go(state, action)),
                    (1, self.go(state,orientations[orientations.index(action)-1])),
                    (1, self.go(state, orientations[(orientations.index(action)+1) % len(orientations)]))]
    def go(self, state, direction):
        state1 = tuple(operator.add(state,direction))
        return state1 if state1 in self.states else state

    def to_grid(self, mapping):
        return list(reversed([[mapping.get((x, y), None)
                               for x in range(self.cols)]
                              for y in range(self.rows)]))

mdp = GridMDP([[0, 0, 0, +10],
                     [0, 0,  0, -10],
                     [0, None, 0, 0]],
                    terminals=[(0, 3), (1, 3)])
def argmin(seq, fn):
    best = seq[0]; best_score = fn(best)
    for x in seq:
        x_score = fn(x)
        if x_score < best_score:
            best, best_score = x, x_score
    return best
def argmax(seq, fn):
    return argmin(seq, lambda x: -fn(x))
def value_iteration(mdp, epsilon=0.001):

    U1 = {s: 0 for s in mdp.states}
    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    while True:
        U = U1.copy()
        delta = 0
        qsa=[]
        for s in mdp.states:
            qsa.append([sum([p * U[s1] for (p, s1) in T(s, a)])
                                        for a in mdp.actions(s)])
           
            U1[s] = R(s) + gamma * max([sum([p * U[s1] for (p, s1) in T(s, a)])
                                        for a in mdp.actions(s)])
            delta = max(delta, abs(U1[s] - U[s]))
        qsa=np.array(qsa)
        
        
        if delta < epsilon * (1 - gamma) / gamma:
            
                return U,qsa

def expected_utility(a, s, U, mdp):
    return sum([p * U[s1] for (p, s1) in mdp.T(s, a)])

def policy_iteration(mdp):
    U = {s: 0 for s in mdp.states}
    pi = {s: random.choice(mdp.actions(s)) for s in mdp.states}
    while True:
        U = policy_evaluation(pi, U, mdp)
        print("The values at every policy update",sum(U.values()))
        unchanged = True
        for s in mdp.states:
            a = max(mdp.actions(s), key=lambda a: expected_utility(a, s, U, mdp))
            if a != pi[s]:
                pi[s] = a
                unchanged = False
        if unchanged:
            return pi,U
def policy_evaluation(pi, U, mdp, k=20):
    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    for i in range(k):
        for s in mdp.states:
            U[s] = R(s) + gamma * sum([p * U[s1] for (p, s1) in T(s, pi[s])])
    return U


U,qsa= value_iteration(mdp)
P,Up =policy_iteration(mdp)
print("1. Values of states at convergence for value iteration for discount factor = 0.5",U)
# print("2.qsa pair for every state", qsa)
# print("3.The optimal values using policy iteration",Up)

('The values at every policy update', 0.0)
('The values at every policy update', 0.0)
('1. Values of states at convergence for value iteration for discount factor = 0.5', {(0, 1): 0.0, (1, 2): 0.0, (3, 2): 19.998779296875, (0, 0): 0.0, (3, 0): 0.0, (3, 1): -19.998779296875, (2, 1): 0.0, (1, 1): 0.0, (2, 0): 0.0, (2, 2): 0.0, (0, 2): 0.0})


5),6) - The below code is when the reward is modified by changing the reward for every state to -1 and the transition is no longer deterministic. As it can be seen, the values except goal states have become negative which inturn encourages the agent to reach goal in lesser number of steps.

In [5]:
import random
import operator
import numpy as np
orientations = [(1,0), (0, 1), (-1, 0), (0, -1)]#The different orientations are right, up,left, down
class MDP:

    def __init__(self, init, actlist, terminals, transitions={}, states=None, gamma=.9):
        if states:
            self.states = states
        else:
            self.states = set()
        self.init = init
        self.actlist = actlist
        self.terminals = terminals
        self.transitions = transitions
        self.gamma = gamma
        self.reward = {}

    def R(self, state):
        return self.reward[state]
    def T(self, state, action):
        if(self.transitions == {}):
            raise ValueError("Transition model is missing")
        else:
            return self.transitions[state][action]
    def actions(self, state):
        if state in self.terminals:
            return [None]
        else:
            return self.actlist

class GridMDP(MDP):

    def __init__(self, grid, terminals, init=(0, 0), gamma=.9):
        grid.reverse()  # because we want row 0 on bottom, not on top
        MDP.__init__(self, init, actlist=orientations,
                     terminals=terminals, gamma=gamma)
        self.grid = grid
        self.rows = len(grid)
        self.cols = len(grid[0])
        for x in range(self.cols):
            for y in range(self.rows):
                self.reward[x, y] = grid[y][x]
                if grid[y][x] is not None:
                    self.states.add((x, y))

    def T(self, state, action):
        if action is None:
            return [(0.0, state)]
        else:
            return [(0.8, self.go(state, action)),
                    (0.1, self.go(state,orientations[orientations.index(action)-1])),
                    (0.1, self.go(state, orientations[(orientations.index(action)+1) % len(orientations)]))]
    def go(self, state, direction):
        state1 = tuple(operator.add(state,direction))
        return state1 if state1 in self.states else state

    def to_grid(self, mapping):
        return list(reversed([[mapping.get((x, y), None)
                               for x in range(self.cols)]
                              for y in range(self.rows)]))

mdp = GridMDP([[-1, -1, -1, +10],
                     [-1, -1,  -1, -10],
                     [-1, None, -1, -1]],
                    terminals=[(0, 3), (1, 3)])
def argmin(seq, fn):
    best = seq[0]; best_score = fn(best)
    for x in seq:
        x_score = fn(x)
        if x_score < best_score:
            best, best_score = x, x_score
    return best
def argmax(seq, fn):
    return argmin(seq, lambda x: -fn(x))
def value_iteration(mdp, epsilon=0.001):

    U1 = {s: 0 for s in mdp.states}
    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    while True:
        U = U1.copy()
        delta = 0
        qsa=[]
        for s in mdp.states:
            qsa.append([sum([p * U[s1] for (p, s1) in T(s, a)])
                                        for a in mdp.actions(s)])
           
            U1[s] = R(s) + gamma * max([sum([p * U[s1] for (p, s1) in T(s, a)])
                                        for a in mdp.actions(s)])
            delta = max(delta, abs(U1[s] - U[s]))
        qsa=np.array(qsa)
        
        
        if delta < epsilon * (1 - gamma) / gamma:
            
                return U,qsa

def expected_utility(a, s, U, mdp):
    return sum([p * U[s1] for (p, s1) in mdp.T(s, a)])

def policy_iteration(mdp):
    U = {s: 0 for s in mdp.states}
    pi = {s: random.choice(mdp.actions(s)) for s in mdp.states}
    while True:
        U = policy_evaluation(pi, U, mdp)
        print("The values at every policy update",sum(U.values()))
        unchanged = True
        for s in mdp.states:
            a = max(mdp.actions(s), key=lambda a: expected_utility(a, s, U, mdp))
            if a != pi[s]:
                pi[s] = a
                unchanged = False
        if unchanged:
            return pi,U
def policy_evaluation(pi, U, mdp, k=20):
    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    for i in range(k):
        for s in mdp.states:
            U[s] = R(s) + gamma * sum([p * U[s1] for (p, s1) in T(s, pi[s])])
    return U


U,qsa= value_iteration(mdp)
P,Up =policy_iteration(mdp)
print("1. Values of states at convergence for value iteration",U)
print("2.qsa pair for every state", qsa)
print("3.The optimal values using policy iteration",Up)

('The values at every policy update', -79.05810108684881)
('The values at every policy update', -88.6697205352709)
('1. Values of states at convergence for value iteration', {(0, 1): -9.999897095698548, (1, 2): -9.999897095698548, (3, 2): 99.99897095698557, (0, 0): -9.999897095698548, (3, 0): -9.999897095698548, (3, 1): -99.99897095698557, (2, 1): -9.999897095698548, (1, 1): -9.999897095698548, (2, 0): -9.999897095698548, (2, 2): -9.999897095698548, (0, 2): -9.999897095698548})
('2.qsa pair for every state', array([[ -9.9998971 ,  -9.9998971 ,  -9.9998971 ,  -9.9998971 ],
       [ -9.9998971 ,  -9.9998971 ,  -9.9998971 ,  -9.9998971 ],
       [ 99.99897096,  99.99897096,  99.99897096,  99.99897096],
       [ -9.9998971 ,  -9.9998971 ,  -9.9998971 ,  -9.9998971 ],
       [ -9.9998971 ,  -9.9998971 ,  -9.9998971 ,  -9.9998971 ],
       [-99.99897096, -99.99897096, -99.99897096, -99.99897096],
       [ -9.9998971 ,  -9.9998971 ,  -9.9998971 ,  -9.9998971 ],
       [ -9.9998971 ,  -9.99989